In [ ]:
%cd ..

In [ ]:
import numpy as np
import random
import os
import keras_tuner as kt
from scipy import signal
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as layers
from tensorflow.keras.layers import GlobalMaxPooling2D, Activation, Dense, Conv1D, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalMaxPooling1D
from tensorflow.keras import optimizers

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras import regularizers



from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from utils.signal import DCFilter, normalize
from utils.visualize import showMe, showHistory, showAvg
from utils.augment import apply_augment

from utils.models.conv_classifier import get_conv_classifier
from utils.models.dense_classifier import get_dense_classifier

from config.armband import *

%load_ext autoreload
%autoreload 2

In [ ]:
path = 'data/Nemes/'
post_fix = ''


# dataset = {}
# for c in settings["classes"]:
#     dataset[c] = []

train_set = {}
for c in settings["classes"]:
    train_set[c] = []

val_set = {}
for c in settings["classes"]:
    val_set[c] = []


train_sessions = ['session_1','session_2', 'session_3']
val_sessions = ['session_0']

#for session in os.listdir(path):
for session in train_sessions:
    for c in settings["classes"]:
        print(f'Processing {os.path.join(path, session, c+post_fix)}')
        data = np.load(os.path.join(path, session, c+post_fix+'.npy'),allow_pickle=True)
        #print(data.shape)
        train_set[c].append(data)

for session in val_sessions:
    for c in settings["classes"]:
        print(f'Processing {os.path.join(path, session, c+post_fix)}')
        data = np.load(os.path.join(path, session, c+post_fix+'.npy'),allow_pickle=True)
        #print(data.shape)
        val_set[c].append(data)

# for c in dataset:
#     dataset[c] = np.vstack(dataset[c])
#     print(f'{c}: {dataset[c].shape}')

for c in train_set:
    train_set[c] = np.vstack(train_set[c])
    print(f'{c}: {train_set[c].shape}')

for c in val_set:
    val_set[c] = np.vstack(val_set[c])
    print(f'{c}: {val_set[c].shape}')

#dataset[class] SHAPE -> (number of samples, number of channels, timesteps)
    

In [ ]:
for c in train_set:
    showAvg(train_set,c)

In [ ]:
# pre_processed = {}
# for c in settings["classes"]:
#     pre_processed[c] = []

# for c in dataset:
#     print(f'PreProcessing {c}')
#     for sample in dataset[c]:
#         pre_processed[c].append(normalize(sample))

# for c in pre_processed:
#     pre_processed[c] = np.array(pre_processed[c])
#     print(f'{c}: {pre_processed[c].shape}')

In [ ]:
train_pre_processed = {}
for c in settings["classes"]:
    train_pre_processed[c] = []

for c in train_set:
    print(f'PreProcessing {c}')
    for sample in train_set[c]:
        train_pre_processed[c].append(normalize(sample))

for c in train_pre_processed:
    train_pre_processed[c] = np.array(train_pre_processed[c])
    print(f'{c}: {train_pre_processed[c].shape}')


val_pre_processed = {}
for c in settings["classes"]:
    val_pre_processed[c] = []

for c in val_set:
    print(f'PreProcessing {c}')
    for sample in val_set[c]:
        val_pre_processed[c].append(normalize(sample))

for c in val_pre_processed:
    val_pre_processed[c] = np.array(val_pre_processed[c])
    print(f'{c}: {val_pre_processed[c].shape}')

In [ ]:
for c in train_pre_processed:
    showAvg(train_pre_processed,c, range = [-1, 1])


In [ ]:
# X = np.concatenate([pre_processed[x] for x in sorted(pre_processed)], 0)
# print(X.shape)
# y = range(len(settings["classes"]))
# y = np.repeat(y, [pre_processed[x].shape[0] for x in sorted(pre_processed)])
# y = to_categorical(y)
# print(y.shape)

In [ ]:
train_X = np.concatenate([train_pre_processed[x] for x in sorted(train_pre_processed)], 0)
print(train_X.shape)
train_y = range(len(settings["classes"]))
train_y = np.repeat(train_y, [train_pre_processed[x].shape[0] for x in sorted(train_pre_processed)])
train_y = to_categorical(train_y)
print(train_y.shape)

val_X = np.concatenate([val_pre_processed[x] for x in sorted(val_pre_processed)], 0)
print(val_X.shape)
val_y = range(len(settings["classes"]))
val_y = np.repeat(val_y, [val_pre_processed[x].shape[0] for x in sorted(val_pre_processed)])
val_y = to_categorical(val_y)
print(val_y.shape)

In [ ]:
# #SHUFFLE DATA
# c = list(zip(X, y))
# random.shuffle(c)
# X,y = zip(*c)
# X = np.array(X)
# y = np.array(y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

In [ ]:
train_X, train_y = apply_augment(train_X, train_y)

In [ ]:
#model = get_dense_classifier(train_X.shape, train_y.shape)
model = get_conv_classifier(train_X.shape, train_y.shape)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10,
    decay_rate=0.99
    )

opt = keras.optimizers.Adam(learning_rate=lr_schedule)
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.001,
    name='categorical_crossentropy'
)
model.compile(optimizer=opt,
          loss=loss,
          metrics=['accuracy']
         )

In [14]:
batch_size = 20
history = model.fit(train_X,
                    train_y,
                    validation_data=(val_X, val_y),
                    batch_size=batch_size,
                    epochs=100,
                    shuffle=True)

showHistory(history)          

KeyboardInterrupt: 

In [ ]:
model.save('models/nemes_09_06')

In [ ]:
val_pred = np.array(model(val_X)).argmax(axis=1)
confusion_matrix(val_y.argmax(axis=1), val_pred)
accuracy_score(val_y.argmax(axis=1), val_pred)